In [2]:
import sys
import urllib.request
import numpy as np
sys.path.insert(0, '/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/bayestme-main/src')
from bayestme import data
from sklearn.model_selection import train_test_split
from sklearn import decomposition
import pandas as pd
from sklearn import decomposition
from bayestme import gene_filtering


In [3]:
import os 
import csv
fls=os.listdir('/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/teixeira_code/flipped_cohort_data/')
fls=sorted(fls)
n_comp = np.array(pd.read_csv("../teixeira_code/n_cell_type_cluster_expression.csv").x)
fout=[]
for k in fls:
    fout.append (k[:2])


In [5]:
def fill_array (arr, fill_val):
    sz=[]
    for x in arr:
        sz.append(len(x))
    maxlen=np.max(sz)
    new=[]
    for x in arr:
        while len(x)<maxlen:
            x=np.append (x,-1)
        new.append (x)
    return new

In [ ]:
p=0
import os
from bayestme import phenotype_selection
from pathlib import Path
from bayestme import deconvolution
import h5py
for f in fls:
    print (f)
    stdata = data.SpatialExpressionDataset.read_count_mat('/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/teixeira_code/flipped_cohort_data/'+f)
    from bayestme import gene_filtering

    stddev_filtered = gene_filtering.select_top_genes_by_standard_deviation(
        stdata, n_gene=1000)

    spot_threshold_filtered = gene_filtering.filter_genes_by_spot_threshold(
        stddev_filtered, spot_threshold=0.95)

    stdata_filtered = gene_filtering.filter_ribosome_genes(spot_threshold_filtered)

    n_genes=len(stdata_filtered.gene_names)
    best_lambda=1000000
    best_n_components=n_comp[p]
    
    deconvolution_result = deconvolution.deconvolve(
            reads=stdata_filtered.reads,
            edges=stdata_filtered.edges,
            n_gene=n_genes,
            n_components=best_n_components,
            lam2=best_lambda,
            n_samples=1000,
            n_burnin=2000,
            n_thin=5,
            bkg=False,
            lda=False)
    out=deconvolution_result.__dict__
    mg=deconvolution.select_marker_genes(deconvolution_result)
    mg=np.array(fill_array(mg,-1))
    with h5py.File('/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/teixeira_code/deconvolution_results_1000_sims_lambda_10e5/'+fout[p]+'.h5', 'w') as h5file:
        for key, item in out.items():
            # note that not all variable types are supported but string and int are
            if type (item)!=int:
                if len (item.shape)>0:
                    if item.shape[0]==1000:
                        h5file [key] = np.mean (item, axis=0)
                    else:
                        h5file [key] = item
            else:
                h5file [key] = item
        h5file['gene_names']=np.array(stdata_filtered.gene_names)
        h5file['marker_genes']=mg
        h5file.close()
    p=p+1



A1.tsv


/Users/joaquimteixeira/GSR_22_Tansey_Banerjee/bayestme-main/src/bayestme/data.py:58: FutureWarning: X.dtype being converted to np.float32 from int64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = anndata.AnnData(counts, obsm={SPATIAL_ATTR: coordinates})


In [1]:
10e6

10000000.0